In [1]:
import numpy as np

from reachy_sdk import ReachySDK

from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory.interpolation import InterpolationMode
import time

In [2]:
reachy = ReachySDK('192.168.1.21')

Check if all the joints seem to be detected.

In [5]:
for j in reachy.joints.values():
    j.torque_limit = 100

## Checking compliance

In [6]:
reachy.joints

<Holder
	<Joint name="l_shoulder_pitch" pos="-4.99" mode="compliant">
	<Joint name="l_shoulder_roll" pos="-1.08" mode="compliant">
	<Joint name="l_arm_yaw" pos="-37.41" mode="compliant">
	<Joint name="l_elbow_pitch" pos="1.54" mode="compliant">
	<Joint name="l_forearm_yaw" pos="-40.62" mode="compliant">
	<Joint name="l_wrist_pitch" pos="-5.58" mode="compliant">
	<Joint name="l_wrist_roll" pos="-18.04" mode="compliant">
	<Joint name="l_gripper" pos="35.12" mode="compliant">
	<Joint name="r_shoulder_pitch" pos="-3.23" mode="compliant">
	<Joint name="r_shoulder_roll" pos="-3.14" mode="compliant">
	<Joint name="r_arm_yaw" pos="-6.90" mode="compliant">
	<Joint name="r_elbow_pitch" pos="-0.57" mode="compliant">
	<Joint name="r_forearm_yaw" pos="-9.01" mode="compliant">
	<Joint name="r_wrist_pitch" pos="1.36" mode="compliant">
	<Joint name="r_wrist_roll" pos="22.73" mode="compliant">
	<Joint name="r_gripper" pos="-41.28" mode="compliant">
	<Joint name="l_antenna" pos="19.50" mode="compliant">

In [15]:
reachy.head.look_at(0.5,0.0,0.0,duration=3.0)

Lift both arms, put them in stiff mode and check if *turn_off_smoothly* works.

In [3]:
reachy.turn_on('reachy')

In [4]:
reachy.turn_off_smoothly('reachy')

## Moving the arms

In [77]:
from scipy.spatial.transform import Rotation as R
import numpy as np

R.from_matrix([
    [0, 0, -1],
    [0, 1, 0],
    [1, 0, 0],
]).as_euler('xyz', degrees=True)


/tmp/ipykernel_32083/2867394349.py:4: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  R.from_matrix([


array([  0.        , -89.99999879,   0.        ])

In [7]:
from scipy.spatial.transform import Rotation as R
import numpy as np
def create_target_xyz_roll_yaw(x, y, z, roll, yaw) :
    y_rot = np.around(R.from_euler('y', np.deg2rad(-90)).as_matrix(), 3)
    #print(y_rot)
    x_rot = np.around(R.from_euler('z', np.deg2rad(roll)).as_matrix(), 3)
    z_rot = np.around(R.from_euler('x', np.deg2rad(yaw)).as_matrix(), 3)
    #print(x_rot)
    total_rot = y_rot@x_rot@z_rot
    #print(total_rot)
    total_rot=np.r_[ total_rot, [np.array([1, 1, 1])] ]
    #print(total_rot)
    total_rot=np.c_[ total_rot, np.array([x, y, z, 1]) ] 
    #print(total_rot)
    return total_rot

In [204]:
print(create_target_xyz_roll_yaw(0.3, 0.15, -0.4, 0, 0))

[[ 0.    0.   -1.    0.3 ]
 [ 0.    1.    0.    0.15]
 [ 1.    0.    0.   -0.4 ]
 [ 1.    1.    1.    1.  ]]


For the left arm:

In [177]:
A = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.15],  
  [1, 0, 0, -0.40],
  [0, 0, 0, 1],  
])

B = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.15],  
  [1, 0, 0, -0.445],
  [0, 0, 0, 1],  
])

In [244]:
A=create_target_xyz_roll_yaw(0.3, 0.125, -0.4, 0, -20)
B=create_target_xyz_roll_yaw(0.3, 0.125, -0.435, 0, -20)

In [59]:
targets = []
targets.append(create_target_xyz_roll_yaw(0.3, 0.20, -0.4, -4, -20))
targets.append(create_target_xyz_roll_yaw(0.3, 0.20, -0.445, -4, -20))
targets.append(create_target_xyz_roll_yaw(0.3, 0.17, -0.4, -4, -20))
targets.append(create_target_xyz_roll_yaw(0.3, 0.17, -0.445, -4, -20))
targets.append(create_target_xyz_roll_yaw(0.3, 0.14, -0.4, -0, -20))
targets.append(create_target_xyz_roll_yaw(0.3, 0.14, -0.445, -0, -20))

In [62]:
# put the joints in stiff mode
try:
    reachy.turn_on('l_arm')

    # use the goto function
    for i in range(6) :
        for t in targets :
            goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(),reachy.l_arm.inverse_kinematics(t) )}, duration=0.5)
            time.sleep(0.3)
    
    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), reachy.l_arm.inverse_kinematics(targets[0]))}, duration=1.0)
    time.sleep(0.2)
    # put the joints back to compliant mode
    # use turn_off_smoothly to prevent the arm from falling hard
    #reachy.turn_off_smoothly('l_arm')
except AttributeError:
    print('Reachy has no left arm!')

In [23]:
# put the joints in stiff mode
try:
    reachy.turn_on('l_arm')

    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), reachy.l_arm.inverse_kinematics(targets[0]))}, duration=1.0)
    time.sleep(0.2)
    # put the joints back to compliant mode
    # use turn_off_smoothly to prevent the arm from falling hard
    #reachy.turn_off_smoothly('l_arm')
except AttributeError:
    print('Reachy has no left arm!')

In [63]:
reachy.turn_off_smoothly('reachy')

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/remi/pollen_code/reachy-sdk/reachy_sdk/reachy_sdk.py", line 203, in _start_sync_in_bg
    loop.run_until_complete(self._sync_loop())
  File "/usr/lib/python3.8/asyncio/base_events.py", line 616, in run_until_complete
    return future.result()
  File "/home/remi/pollen_code/reachy-sdk/reachy_sdk/reachy_sdk.py", line 258, in _sync_loop
    await asyncio.gather(
  File "/home/remi/pollen_code/reachy-sdk/reachy_sdk/reachy_sdk.py", line 226, in _get_stream_sensor_loop
    async for state_update in stub.StreamSensorStates(stream_req):
  File "/home/remi/.virtualenvs/reachy/lib/python3.8/site-packages/grpc/aio/_call.py", line 326, in _fetch_stream_responses
    await self._raise_for_status()
  File "/home/remi/.virtualenvs/re